# Some preparation

In [2]:
%cd /workspace/ToxVidLM_ACL_2024

/workspace/ToxVidLM_ACL_2024


In [2]:
!pwd

/workspace/ToxVidLM_ACL_2024


## Packages Installation

In [4]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 26.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 30.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 30.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 32.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 30.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 21.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install peft==0.6.2

In [6]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.1 MB/s eta 0:00:00


In [8]:
# Add this cell before the train cell
!pip uninstall torch torchvision torchaudio -y
!pip install torch==1.12.1+cu102 torchvision==0.13.1+cu102 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu102

# Verify installation
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"PyTorch version: {torch.__version__}")

Found existing installation: torch 2.0.0
Uninstalling torch-2.0.0:
  Successfully uninstalled torch-2.0.0
Found existing installation: torchvision 0.15.1
Uninstalling torchvision-0.15.1:
  Successfully uninstalled torchvision-0.15.1
Found existing installation: torchaudio 2.0.1
Uninstalling torchaudio-2.0.1:
  Successfully uninstalled torchaudio-2.0.1
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu102
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 52.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 58.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 9.0 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.6.2 requires torch>=1.13.0, but you have torch 1.12.1+cu102 which is incompatible.
CUDA available: True
CUDA ver

In [1]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"PyTorch version: {torch.__version__}")

CUDA available: True
CUDA version: 10.2
PyTorch version: 1.12.1+cu102


# Download Dataset

In [ ]:
# Install gdown first
!pip install gdown


In [4]:
%cd /workspace/

/workspace


In [ ]:


# For a public folder, you can try:
import gdown
import os

# Create download directory
os.makedirs("g_downloaded_data", exist_ok=True)

In [8]:
!echo "final_data/" >> .gitignore
!echo "g_downloaded_data/" >> .gitignore
!echo "downloaded_data/" >> .gitignore

In [2]:


# Download the entire folder (this works for some public folders)
folder_id = "1fddccfJkYZifbRF9zrhYlmX4IAMICnwb"
gdown.download_folder(f"https://drive.google.com/drive/folders/{folder_id}", output="g_downloaded_data/", quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 1Tj31_sm-gLGVtr5ZeqGnmfF_0OxVCRIz classification_final_data-20250630T215505Z-1-001.zip
Processing file 1TOHalYk6aZQBOrg1PdPYzaHA0KlobgVX classification_final_data-20250630T215505Z-1-002.zip
Processing file 1QhdmWLFPdQFGI3myJ2QpxzT9rXp6cU46 classification_final_data-20250630T215505Z-1-003.zip


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1Tj31_sm-gLGVtr5ZeqGnmfF_0OxVCRIz
From (redirected): https://drive.google.com/uc?id=1Tj31_sm-gLGVtr5ZeqGnmfF_0OxVCRIz&confirm=t&uuid=d8cf1231-a1b8-47db-98c7-87ec33f3444b
To: /workspace/ToxVidLM_ACL_2024/g_downloaded_data/ToxVid/classification_final_data-20250630T215505Z-1-001.zip
100%|██████████| 2.15G/2.15G [02:52<00:00, 12.5MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1TOHalYk6aZQBOrg1PdPYzaHA0KlobgVX
From (redirected): https://drive.google.com/uc?id=1TOHalYk6aZQBOrg1PdPYzaHA0KlobgVX&confirm=t&uuid=df5ed501-d21b-4124-8f23-99997f848f18
To: /workspace/ToxVidLM_ACL_2024/g_downloaded_data/ToxVid/classification_final_data-20250630T215505Z-1-002.zip
100%|██████████| 2.15G/2.15G [02:51<00:00, 12.5MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1QhdmWLFPdQFGI3myJ2QpxzT9rXp6cU46
From

['g_downloaded_data/ToxVid/classification_final_data-20250630T215505Z-1-001.zip',
 'g_downloaded_data/ToxVid/classification_final_data-20250630T215505Z-1-002.zip',
 'g_downloaded_data/ToxVid/classification_final_data-20250630T215505Z-1-003.zip']

In [4]:
import os
import zipfile
download_dir = "g_downloaded_data/ToxVid"
# List all files in the download directory
for filename in os.listdir(download_dir):
    if filename.endswith(".zip"):
        zip_path = os.path.join(download_dir, filename)
        print(f"Extracting {zip_path}...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(download_dir)      
print("Extraction complete.")

Extracting g_downloaded_data/ToxVid/classification_final_data-20250630T215505Z-1-001.zip...


Extracting g_downloaded_data/ToxVid/classification_final_data-20250630T215505Z-1-002.zip...
Extracting g_downloaded_data/ToxVid/classification_final_data-20250630T215505Z-1-003.zip...
Extraction complete.


In [5]:
!mv g_downloaded_data/ToxVid/classification_final_data final_data

In [7]:
import shutil
import os

source_folder = "final_data/classification_final_data"
destination_folder = "final_data"

# Check if source folder exists
if os.path.exists(source_folder):
    # List all files in the source folder
    files = os.listdir(source_folder)
    
    # Move each file to the destination folder
    for file in files:
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(destination_folder, file)
        shutil.move(source_path, destination_path)
    
    print("Files moved successfully.")
else:
    print("Source folder does not exist.")

Files moved successfully.


# Git

In [11]:
# Remove folders from Git tracking (but keep them locally)
!git rm -r --cached final_data/
!git rm -r --cached g_downloaded_data/
!git rm -r --cached downloaded_data/
!git rm -r --cached data/

fatal: pathspec 'final_data/' did not match any files


fatal: pathspec 'g_downloaded_data/' did not match any files
fatal: pathspec 'downloaded_data/' did not match any files


In [13]:
# Add the .gitignore changes
!git add .gitignore

# Commit the changes
!git commit -m "Add data folders to .gitignore"

[main 521de73] Add data folders to .gitignore
 3 files changed, 878 insertions(+)
 create mode 100644 .gitignore
 delete mode 100644 final_data/final_data.txt.txt
 create mode 100644 train_test.ipynb


In [ ]:
!git add .
!git commit -m "Your descriptive commit message"
!git push origin main

# Train

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


In [19]:
# Set CUDA memory allocation configuration to reduce fragmentation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:256'

In [6]:
# From train.py

import torch
import torch.nn as nn
from tokenizers import AddedToken
from transformers import CLIPModel, VideoMAEModel, Wav2Vec2Model, VideoMAEConfig, CLIPConfig, Wav2Vec2Config, XLMRobertaConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSeq2SeqLM
from model.additional_modules import LSTM_fc, FC_head, Gate_Attention
from argparse import Namespace 
from model.model import Multimodal_LLM
from data.dataset import CustomDataset
from iteration import train_model, train_one_epoch, validate
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, AlbertTokenizer, XLMRobertaTokenizerFast, PreTrainedTokenizerFast #only for gpt2 and assign values
from transformers import GPT2Model, BertModel, AlbertModel, XLMRobertaModel
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import os

tasks_bool = {"offensive" : True, "offensive_level": True, "sentiment" : True}
tasks = []
name = "gpt2_vidmae_whisper_"

for k, v in tasks_bool.items():
    if tasks_bool[k]:
        tasks.append(k)
        name += k + "_"
        
config = Namespace(
    file_name=name + "0",
    device=torch.device("cuda:0"),
    tokenizer_path="ckpts",
    tasks = tasks,
    offensive_bool = tasks_bool["offensive"],
    offensive_level_bool = tasks_bool["offensive_level"],
    sentiment_bool = tasks_bool["sentiment"],
    video_encoder="MCG-NJU/videomae-base",
    audio_encoder="openai/whisper-small",
    lstm_or_conv = False,
    image_conv_kernel=23,
    image_conv_stride=3,
    image_conv_padding=8,
    video_conv_kernel=36,
    video_conv_stride=24,
    video_conv_padding=0,
    audio_conv_kernel=50,
    audio_conv_stride=23,
    audio_conv_padding=1,
    llm_embed_dim=768,
    llm_output_dim=768,
    attn_dropout=0.1,
    is_add_bias_kv=True,
    is_add_zero_attn=True,
    attention_heads=8,
    image_dim=768,
    video_dim=768,
    audio_dim=768,
    image_seq_len=197,
    video_seq_len=1568,
    audio_seq_len=1500,
    min_mm_seq_len=64,
    lstm_num_layers=1,
    tokenizer_max_len=128,
    add_pooling = False,
    train=True,
    directory = "checkpoints/",
    results_directory = "results/"
)




/workspace/.conda/lib/python3.8/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/workspace/.conda/lib/python3.8/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [ ]:
df = pd.read_csv("final_data/final_processed_data_one_hot.csv")
df_train_val, df_test = train_test_split(df, test_size=0.4, random_state=28703)
df_train, df_val = train_test_split(df_train_val, test_size=0.4, random_state=28703)

num_epochs = 1
patience = 10
batch_size = 2

#for roberta
tokenizer = XLMRobertaTokenizerFast.from_pretrained("l3cube-pune/hing-roberta")
model = XLMRobertaModel.from_pretrained("l3cube-pune/hing-roberta", torch_dtype=torch.float32)


model = Multimodal_LLM(batch_size=batch_size, config=config, tokenizer=tokenizer, adapter_llm=model)

train_ds = CustomDataset(dataframe=df_train, train=True, tokenizer=tokenizer)
val_ds = CustomDataset(df_val, train=True, tokenizer=tokenizer)
test_ds = CustomDataset(df_test, train=False, tokenizer=tokenizer)

train_dataloader = DataLoader(train_ds, batch_size=batch_size, num_workers=0, shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=batch_size, num_workers=0)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, num_workers=0)


train_model(model, train_dataloader, val_dataloader, config, num_epochs, "offensive", "f1", devices=None)

In [5]:
!python train.py

/workspace/.conda/lib/python3.8/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/workspace/.conda/lib/python3.8/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Some weights of XLMRobertaModel were not initialized from the model checkpoint at l3cube-pune/hing-roberta and are newly initialized: ['roberta.pooler.dense.bias', 'rob

In [1]:
!python test.py

/workspace/.conda/lib/python3.8/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/workspace/.conda/lib/python3.8/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


Some weights of XLMRobertaModel were not initialized from the model checkpoint at l3cube-pune/hing-roberta and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Traceback (most recent call last):
  File "test.py", line 112, in <module>
    state_dict = torch.load(checkpoint_path, map_location=config.device)
  File "/workspace/.conda/lib/python3.8/site-packages/torch/serialization.py", line 791, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/workspace/.conda/lib/python3.8/site-packages/torch/serialization.py", line 271, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/workspace/.conda/lib/python3.8/site-packages/torch/serialization.py", line 252, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/model_save.pth'


# For debug

In [3]:
import torch
import gc

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print(f"GPU memory cleared. Available: {torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)} bytes")

GPU memory cleared. Available: 34072559616 bytes


In [5]:
import torch
import torch.nn as nn
from tokenizers import AddedToken
from transformers import CLIPModel, VideoMAEModel, Wav2Vec2Model, VideoMAEConfig, CLIPConfig, Wav2Vec2Config, XLMRobertaConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSeq2SeqLM
from model.additional_modules import LSTM_fc, FC_head, Gate_Attention
from argparse import Namespace 
from model.model import Multimodal_LLM
from data.dataset import CustomDataset
from iteration import train_model, train_one_epoch, validate
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, AlbertTokenizer, XLMRobertaTokenizerFast, PreTrainedTokenizerFast #only for gpt2 and assign values
from transformers import GPT2Model, BertModel, AlbertModel, XLMRobertaModel
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import os

tasks_bool = {"offensive" : True, "offensive_level": True, "sentiment" : True}
tasks = []
name = "gpt2_vidmae_whisper_"

for k, v in tasks_bool.items():
    if tasks_bool[k]:
        tasks.append(k)
        name += k + "_"
        
config = Namespace(
    file_name=name + "0",
    device=torch.device("cuda:1"),
    tokenizer_path="ckpts",
    tasks = tasks,
    offensive_bool = tasks_bool["offensive"],
    offensive_level_bool = tasks_bool["offensive_level"],
    sentiment_bool = tasks_bool["sentiment"],
    video_encoder="MCG-NJU/videomae-base",
    audio_encoder="openai/whisper-small",
    lstm_or_conv = False,
    image_conv_kernel=23,
    image_conv_stride=3,
    image_conv_padding=8,
    video_conv_kernel=36,
    video_conv_stride=24,
    video_conv_padding=0,
    audio_conv_kernel=50,
    audio_conv_stride=23,
    audio_conv_padding=1,
    llm_embed_dim=768,
    llm_output_dim=768,
    attn_dropout=0.1,
    is_add_bias_kv=True,
    is_add_zero_attn=True,
    attention_heads=8,
    image_dim=768,
    video_dim=768,
    audio_dim=768,
    image_seq_len=197,
    video_seq_len=1568,
    audio_seq_len=1500,
    min_mm_seq_len=64,
    lstm_num_layers=1,
    tokenizer_max_len=128,
    add_pooling = False,
    train=True,
    directory = "checkpoints/",
    results_directory = "results/"
)


/workspace/ToxVidLM_ACL_2024/.conda/lib/python3.8/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [6]:
json_save_path = config.results_directory + config.file_name + ".json"

In [8]:
import json
history={"a":"b"}
with open(json_save_path, 'w') as json_file:
    json.dump(history, json_file)

In [ ]:
# This cell was cleaned up - the train_model function is already imported from iteration.py
# No need to redefine it here